# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [5]:
# boston is dictionary with key: data & target
boston.target[0]

24.0

In [6]:
boston_features.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,0.542096,1.0,296.0,15.3,1.000000,-1.275260
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,0.623954,2.0,242.0,17.8,1.000000,-0.263711
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,0.623954,2.0,242.0,17.8,0.989737,-1.627858
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,0.707895,3.0,222.0,18.7,0.994276,-2.153192
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,0.707895,3.0,222.0,18.7,1.000000,-1.162114


In [7]:
X = boston_features[['CHAS', 'RM', 'DIS', 'B', 'LSTAT']]
y = pd.DataFrame(boston.target,columns = ['target'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [8]:
from sklearn.model_selection import train_test_split

# returns list of 4 dataframes : 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)



In [9]:
len(X_train), len(X_test), len(y_train), len(y_test)

(404, 102, 404, 102)

In [10]:
102/(102+404)    # length of test set

0.2015810276679842

Fit the model and apply the model to the make test set predictions

In [11]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train,y_train)   # Fitting on Training data
y_hat_test = linreg.predict(X_test)   # Predicting on Test data
y_hat_test[0]

array([28.39706293])

Calculate the residuals and the mean squared error

In [12]:
from sklearn.metrics import mean_squared_error
test_resid = y_hat_test - y_test
test_mse = mean_squared_error(y_test, y_hat_test)
test_mse

20.397020195518206

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [13]:
def kfolds(data, k):
    # list of folds
    folds = []
    
    # Force data as pandas dataframe
    df = pd.DataFrame(data)
    length = len(data)
    
    # size of each fold
    fold_size = length//k
    rest = length % k
    
    start_obs = 0
    for fold_n in range(0,k+1):  # k+1 if data left after k folds
        if fold_n <= rest:
            #Fold Size will be 1 larger to account for leftovers
            fold =  data.iloc[start_obs : start_obs+fold_size+1] 
            folds.append(fold)
            start_obs +=  fold_size + 1
        else:
            fold =  data.iloc[start_obs : start_obs+fold_size] 
            folds.append(fold)
            start_obs +=  fold_size
            
    return folds 

### Apply it to the Boston Housing Data

In [14]:
boston_data = pd.concat([X,y],axis =1)
boston_data.head()

,CHAS,RM,DIS,B,LSTAT,target
0,0.0,6.575,0.542096,1.000000,-1.275260,24.0
1,0.0,6.421,0.623954,1.000000,-0.263711,21.6
2,0.0,7.185,0.623954,0.989737,-1.627858,34.7
3,0.0,6.998,0.707895,0.994276,-2.153192,33.4
4,0.0,7.147,0.707895,1.000000,-1.162114,36.2


In [15]:
bos_folds = kfolds (boston_data, k = 5)

In [23]:
for fold in bos_folds:
    print(len(fold))

102
102
101
101
100
0


### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [17]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(bos_folds) if i!=n])
    test = bos_folds[n]
    # Fit a linear regression model
    linreg.fit(train[X.columns], train[y.columns])
    
    # Evaluate Train and Test errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    
    # Errors
    train_residuals = y_hat_train - train[y.columns]
    test_residuals = y_hat_test - test[y.columns]
    
    
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))
print(train_errs)
print(test_errs)


[target    24.195577
dtype: float64, target    22.744595
dtype: float64, target    20.013708
dtype: float64, target    15.082497
dtype: float64, target    22.533449
dtype: float64]
[target    13.405145
dtype: float64, target    18.924231
dtype: float64, target    35.418659
dtype: float64, target    59.618318
dtype: float64, target    25.294289
dtype: float64]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [18]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")
cv_5_results


array([-13.40514492, -17.4440168 , -37.03271139, -58.27954385,
       -26.09798876])

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [19]:
# Average MSE for 5-fold cross-validation
-np.mean(cv_5_results)

30.451881143540298

In [22]:
# Train-test-split MSE
test_mse

20.397020195518206

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!